In [ ]:
import requests
import pandas as pd
import pysolr

In [ ]:
url = "http://b2find.eudat.eu/solr/"
# url = "http://eudat8.dkrz.de/solr/"


In [ ]:
import time
years = year = 60 * 60 * 24 * 365
year1epochsec = 62135600400
now = year1epochsec + int(time.time())

facet = "extras_TempCoverageBegin"
#facet = "extras_TempCoverageEnd"
min = now-10*years
max = now+10*years
num_intervals=100
interval=int((max-min)/num_intervals)
interval

In [ ]:
solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
    'facet': 'false',
    'indent': 'false',
    'fl': 'id',
    'group': 'true',
    'group.limit': 1,
    'group.query': []
}

In [ ]:
query = 'extras_TempCoverageBegin:[* TO {end}] AND extras_TempCoverageEnd:[{start} TO *]'

In [ ]:
x = []
y = []

for i in range(num_intervals):
    start = int(min + i * interval)
    end = int(min + (i+1) * interval)
    group_query = query.format(start=start, end=end)
    solr_params["group.query"].append(group_query)
    x.append(start-year1epochsec)

In [ ]:
solr = pysolr.Solr(url)

In [ ]:
results = solr.search(**solr_params)
results

In [ ]:
for val in results.grouped.values():
    num_found = val['doclist']['numFound']
    y.append(num_found)

In [ ]:
df = pd.DataFrame(list(zip(x, y)),
               columns =['start', 'counts'])
df

In [ ]:
df["start"]=pd.to_datetime(df["start"],unit="s")
df

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df)

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(df.start.loc[0], df.start.loc[20]))

p.line('start', 'counts', source=source)
p.yaxis.axis_label = 'datasets'

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('start', 'counts', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))